<a href="https://colab.research.google.com/github/Lineage-Reconstruction-DREAM-Challenge/C2_C3_training_data_performance/blob/master/FT2_C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Evaluating the running time of FT2 on a dataset of 1,000 cells with 200 targets

In [0]:
!rm -f FastTree.c
!wget http://www.microbesonline.org/fasttree/FastTree.c
!gcc -DNO_SSE -O3 -finline-functions -funroll-loops -Wall -o FastTree FastTree.c -lm

--2020-06-07 22:33:53--  http://www.microbesonline.org/fasttree/FastTree.c
Resolving www.microbesonline.org (www.microbesonline.org)... 128.32.248.24
Connecting to www.microbesonline.org (www.microbesonline.org)|128.32.248.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 395543 (386K) [text/plain]
Saving to: ‘FastTree.c’

FastTree.c          100%[===================>] 386.27K   683KB/s    in 0.6s    

2020-06-07 22:33:54 (683 KB/s) - ‘FastTree.c’ saved [395543/395543]



In [0]:
!wget -O C2_test.tsv https://s3.msi.umn.edu/gongx030/rlib/C2_C3_training_data_performance/C2_test.tsv
!wget -O unified_transition_table_for_fastree.mat https://s3.msi.umn.edu/gongx030/rlib/C2_C3_training_data_performance/dream_examples/unified_transition_table_for_fastree.mat

--2020-06-07 22:34:02--  https://s3.msi.umn.edu/gongx030/rlib/C2_C3_training_data_performance/C2_test.tsv
Resolving s3.msi.umn.edu (s3.msi.umn.edu)... 128.101.189.144
Connecting to s3.msi.umn.edu (s3.msi.umn.edu)|128.101.189.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 421892 (412K) [text/plain]
Saving to: ‘C2_test.tsv’

C2_test.tsv         100%[===================>] 412.00K  2.40MB/s    in 0.2s    

2020-06-07 22:34:02 (2.40 MB/s) - ‘C2_test.tsv’ saved [421892/421892]

--2020-06-07 22:34:03--  https://s3.msi.umn.edu/gongx030/rlib/C2_C3_training_data_performance/dream_examples/unified_transition_table_for_fastree.mat
Resolving s3.msi.umn.edu (s3.msi.umn.edu)... 128.101.189.144
Connecting to s3.msi.umn.edu (s3.msi.umn.edu)|128.101.189.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4802 (4.7K) [text/plain]
Saving to: ‘unified_transition_table_for_fastree.mat’

unified_transition_ 100%[===================>]   4.69K  --.-KB/s 

In [0]:
!pip install git+https://github.com/jeetsukumaran/DendroPy.git

  Cloning https://github.com/jeetsukumaran/DendroPy.git to /tmp/pip-req-build-r3uhlbmt
  Running command git clone -q https://github.com/jeetsukumaran/DendroPy.git /tmp/pip-req-build-r3uhlbmt
  Created wheel for DendroPy: filename=DendroPy-4.4.0-cp36-none-any.whl size=450883 sha256=d2afcdf1cf908e060e3bbd3f76fdd30108181bac3fe317591fcfa1654e1fb47a
  Stored in directory: /tmp/pip-ephem-wheel-cache-dm7klmoh/wheels/be/75/21/686ec0b11dc2e8e4824cd5d73ba2f71e438d93decee8b6f1b4
Successfully built DendroPy


In [0]:
import os
import sys
import pandas as pd
import dendropy
import subprocess
import time
import numpy as np

In [0]:
df = pd.read_table('C2_test.tsv', sep = '\t')

In [0]:
symbol_map = {
    0.0:'A',
    '0':'A',
    'A':'R',
    'B':'N',
    'C':'D',
    'D':'C',
    'E':'Q',
    'F':'E',
    'G':'G',
    'H':'H',
    'I':'I',
    'J':'L',
    'K':'K',
    'L':'M',
    'M':'F',
    'N':'P',
    'O':'S',
    'P':'T',
    'Q':'W',
    'R':'Y',
    'S':'-', # Skipping rare characters as we're running out of standard amino acids
    'T':'-',
    'U':'-',
    'V':'-',
    '-':'V', # Inherited deletion is treated as charcter as it carries information (unlike acquisition dropout)
}

In [0]:
for col in df.columns:
    df[col] = df[col].map(symbol_map)
df.fillna("-",inplace=True)
normalized_newick_path = 'input.fasta'
with open(normalized_newick_path, 'w') as f:
    for sr, row in df.iterrows():
        f.write('>{}\r\n{}\r\n'.format(sr, ''.join(row.values)))

In [0]:
fasttree_path = "./FastTree"
fasttree_transition_table = 'unified_transition_table_for_fastree.mat'

def run_fasttree(newick_output_file, mutations_fasta_path):
    subprocess.call([fasttree_path, 
                 '-pseudo', 
                 '-trans', fasttree_transition_table,
                 '-out', newick_output_file, 
                 mutations_fasta_path])
    good_tree = dendropy.Tree.get_from_path(
                        newick_output_file,
                    "newick")
    return good_tree

In [0]:
newick_output_file = 'FT2.newick'
t0 = time.time()
rec_tree = run_fasttree(newick_output_file, normalized_newick_path)
print('Time elasped: ' + str(np.round(time.time() - t0, 3)) + " seconds")

Time elasped: 38.005 seconds
